# Image Classification API
This notebook contains the end-to-end workflow for training and deploying an image classification model.

In [ ]:
# Install necessary dependencies
!pip install tensorflow fastapi uvicorn numpy pillow torch torchvision

## Data Preprocessing

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define data augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# Load dataset
dataset_path = "./dataset"
train_data = datagen.flow_from_directory(
    dataset_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='training'
)
val_data = datagen.flow_from_directory(
    dataset_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='validation'
)

## Model Training

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(10, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 10
model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# Save model
model.save('./model/mobilenetv2_saved_model')

## Model Deployment (FastAPI)

In [ ]:
from fastapi import FastAPI, UploadFile, File
from io import BytesIO
from PIL import Image
import uvicorn

# Initialize FastAPI
app = FastAPI()

# Load trained model
model = tf.keras.models.load_model('./model/mobilenetv2_saved_model')
class_names = {0: 'Class_A', 1: 'Class_B', 2: 'Class_C'}

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    image = Image.open(BytesIO(contents)).convert("RGB")
    image = image.resize(IMG_SIZE)
    image_array = np.array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    predictions = model.predict(image_array)
    predicted_class = np.argmax(predictions)
    return {"class": class_names.get(predicted_class, "Unknown"), "confidence": float(np.max(predictions))}

## Running the API

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)